# Make For Loops 10x Faster with Multithreading
Let's start with an example. We will fake and simulate a time-consuming task. We will use a Python script that processes a list of numbers by squaring each number using a for loop:. This script processes each number in the list sequentially, taking 1 second per number due to the `time.sleep(1)` call in the `square_number` function. Total execution takes 10.1 seconds.

In [1]:
import time

# List of numbers to process
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Function to square a number
def square_number(number):
    time.sleep(1)  # Simulate a time-consuming task
    return number * number

# Using a for loop to process each number
squared_numbers = []
start_time = time.time()
for number in numbers:
    squared_numbers.append(square_number(number))

end_time = time.time()

print("Squared numbers:", squared_numbers)
print("Time taken:", end_time - start_time, "seconds")

Squared numbers: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Time taken: 10.087681770324707 seconds


# Example Optimize Multithreading
Next, we'll optimize this with a multithreading approach to improve the processing time. To optimize the above example using multithreading, we can use Python's concurrent.futures module, which provides a high-level interface for asynchronously executing callables. In this optimized script, we use `ThreadPoolExecutor` to create a pool of threads. The `executor.map` function distributes the `square_number` function across the threads, processing the numbers in parallel. By setting `max_workers` to 5, we allow up to 5 threads to run concurrently, which should significantly reduce the total processing time.

In [2]:
import time
from concurrent.futures import ThreadPoolExecutor

# List of numbers to process
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Function to square a number
def square_number(number):
    time.sleep(1)  # Simulate a time-consuming task
    return number * number

# Using ThreadPoolExecutor for multithreading
squared_numbers = []
start_time = time.time()

with ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(square_number, numbers)

# Collect the results
squared_numbers = list(results)

end_time = time.time()

print("Squared numbers:", squared_numbers)
print("Time taken:", end_time - start_time, "seconds")

Squared numbers: [1, 4, 9, 16, 25, 36, 49, 64, 81, 100]
Time taken: 1.020650863647461 seconds


# Pro Tip - Use Decorators
Decorators can be used to add multithreading to functions in a more elegant and reusable way. A decorator is a function that takes another function and extends its behavior without explicitly modifying it.

In [3]:
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# Decorator to add multithreading
def multithreaded(max_workers=5):
    def decorator(func):
        def wrapper(*args, **kwargs):
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                future_to_args = {executor.submit(func, arg): arg for arg in args[0]}
                results = []
                for future in as_completed(future_to_args):
                    arg = future_to_args[future]
                    try:
                        result = future.result()
                    except Exception as exc:
                        print(f'{arg} generated an exception: {exc}')
                    else:
                        results.append(result)
                return results
        return wrapper
    return decorator

# Function to square a number
@multithreaded(max_workers=5)
def square_number(number):
    time.sleep(1)  # Simulate a time-consuming task
    return number * number

# List of numbers to process
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Using the decorated function
start_time = time.time()
squared_numbers = square_number(numbers)
end_time = time.time()

print("Squared numbers:", squared_numbers)
print("Time taken:", end_time - start_time, "seconds")

Squared numbers: [4, 1, 25, 9, 16, 49, 36, 64, 81, 100]
Time taken: 2.01415753364563 seconds
